<h1 align="left">
  <picture>
    <source media="(prefers-color-scheme: dark)" srcset="../assets/logo/123D_logo_transparent_white.svg" width="750">
    <source media="(prefers-color-scheme: light)" srcset="../assets/logo/123D_logo_transparent_black.svg" width="750">
    <img alt="Logo" src="assets/logo/123D_logo_transparent_black.svg" width="1000">
  </picture>
  <h2 align="left">123D: Visualization Tutorial</h1>
</h1>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from py123d.api import MapAPI, SceneAPI, SceneFilter
from py123d.api.scene.arrow.arrow_scene_builder import ArrowSceneBuilder
from py123d.common.multithreading.worker_parallel import SingleMachineParallelExecutor

## 3.1 Download Demo Logs

In [ ]:
# TODO

## 3.2 Create Scenes by filtering the datasets

As in other tutorials, we first query some scenes fro visualization. 

In [ ]:
from py123d.datatypes.sensors.pinhole_camera import PinholeCameraType

scene_filter = SceneFilter(
    split_names=None,
    duration_s=7.0,  # No duration means that the scene will include the complete log.
    timestamp_threshold_s=0.0,
    shuffle=True,
    map_api_required=False,  # Only include scenes/logs with an available map API.
    pinhole_camera_types=[PinholeCameraType.PCAM_F0],
)
worker = SingleMachineParallelExecutor()
scenes = ArrowSceneBuilder().get_scenes(scene_filter, worker)
print(f"Found {len(scenes)} scenes.")

## 3.3 Matplotlib


### 3.3.1 Plots in 2D


In [ ]:
from py123d.visualization.matplotlib.plots import add_scene_on_ax

scene: SceneAPI = np.random.choice(scenes)
map_api: MapAPI = scene.get_map_api()
fig, ax = plt.subplots(figsize=(10, 10))
add_scene_on_ax(ax, scene, iteration=0, radius=80)
plt.show()

In [ ]:
from pathlib import Path

from py123d.visualization.matplotlib.plots import render_scene_animation

save_path = Path("./visualization")
save_path.mkdir(parents=True, exist_ok=True)

fps = 1 / scene.log_metadata.timestep_seconds

render_scene_animation(
    scene=scene,
    output_path=save_path,
    start_idx=0,
    end_idx=None,
    step=1,
    fps=fps * 2,  # Let's speed it up a bit
    dpi=300,
    format="mp4",
    radius=80,
)

### 3.3.2 Plots of Cameras

In [ ]:
from py123d.visualization.matplotlib.camera import add_pinhole_camera_ax

iteration = 0
available_pinhole_cameras = scene.available_pinhole_camera_types
if len(available_pinhole_cameras) > 1:
    pinhole_camera_type = np.random.choice(available_pinhole_cameras)
    pinhole_camera = scene.get_pinhole_camera_at_iteration(iteration=iteration, camera_type=pinhole_camera_type)

    fig, ax = plt.subplots(figsize=(8, 6))
    add_pinhole_camera_ax(ax, pinhole_camera)
    ax.set_title(f"{pinhole_camera_type} at Iteration {iteration}")
    plt.show()

In [ ]:
from py123d.visualization.matplotlib.camera import add_box_detections_to_camera_ax

iteration = 0
available_pinhole_cameras = scene.available_pinhole_camera_types
if len(available_pinhole_cameras) > 1:
    pinhole_camera_type = np.random.choice(available_pinhole_cameras)
    pinhole_camera = scene.get_pinhole_camera_at_iteration(iteration=iteration, camera_type=pinhole_camera_type)
    box_detections = scene.get_box_detections_at_iteration(iteration=iteration)
    ego_state = scene.get_ego_state_at_iteration(iteration=iteration)

    fig, ax = plt.subplots(figsize=(8, 6))
    add_box_detections_to_camera_ax(ax, pinhole_camera, box_detections, ego_state)
    ax.set_title(f"{pinhole_camera_type} at Iteration {iteration}")
    plt.show()

In [ ]:
from py123d.visualization.matplotlib.camera import add_lidar_to_camera_ax

available_pinhole_cameras = scene.available_pinhole_camera_types
available_lidars = scene.available_lidar_types

iteration = 20
if len(available_pinhole_cameras) > 1 and len(available_lidars) > 0:
    pinhole_camera_type = np.random.choice(available_pinhole_cameras)
    pinhole_camera = scene.get_pinhole_camera_at_iteration(iteration=iteration, camera_type=pinhole_camera_type)
    lidar_type = np.random.choice(available_lidars)
    lidar = scene.get_lidar_at_iteration(iteration=iteration, lidar_type=lidar_type)

    fig, ax = plt.subplots(figsize=(8, 6))
    add_lidar_to_camera_ax(ax, pinhole_camera, lidar)
    ax.set_title(f"{pinhole_camera_type} & {lidar_type} at Iteration {iteration}")
    plt.show()

## 3.4. Viser

Can also be run in the `example/01_viser.py` or by running 

```sh
py123d-viser scene_filter=...
```

in your terminal. By default, you can open viewer via: [`http://localhost:8080`](http://localhost:8080 )


In [ ]:
from py123d.visualization.viser.viser_viewer import ViserViewer

ViserViewer(scenes)